In [1]:
import cv2
import numpy as np
import os
import gc
import matplotlib.pyplot as pyp

In [2]:
# Initialize global variables and enable garbage collection
# This is gonna be a huge job
gc.enable()
dir = "/home/alice/Jupyter/OpenCV/images/"
out = "/home/alice/Jupyter/OpenCV/bitwise_edge_output/"

buoy = cv2.imread(dir+"redbuoy.png")
img = []
original = []
max_size = len(os.listdir(dir))
iteration = 0
arr_size = 400

In [3]:
# Read in images and pass them to the transform function
def iterator():
    global max_size
    for i in range(0,max_size-2):
        img.append(cv2.imread(dir + "left" + str(i).zfill(4) + ".jpg"))
        original.append(img[i % arr_size])
        
        transform_stable(i)
    return True

In [4]:
#Process image by deleting background information, blurring, doing a sobel transform, and thresholding
def transform_stable(i):
    global iteration
    global original
    if i > 0 and (len(img) % arr_size == 0 or len(img) + arr_size * iteration == max_size):        
        for j in range(0, len(img)):
            im = np.array(img[j])[0:500][0:500]
            im = delete_blue(im)
            im = np.uint8(im)
            im = cv2.cvtColor(im, cv2.COLOR_HSV2BGR)
            #im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
            #im = cv2.Canny(im, 100, 200)
            #im = cv2.GaussianBlur(im, (11,11), 0)
            #im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
            #im = cv2.Sobel(im, cv2.CV_8U, 1, 0 , 1, 0, cv2.BORDER_DEFAULT)
            #im = cv2.threshold(im, 70, 255, cv2.THRESH_BINARY)[1]
            try:
                im = compare(buoy, im)
            except:
                print(j+iteration*arr_size)
            write(im, j)
            
        reset()
        iteration = iteration + 1
        return True
    return False

In [5]:
#Process image by deleting background information, blurring, doing a sobel transform, and thresholding
def transform(i):
    global iteration
    global original
    if i > 0 and (len(img) % arr_size == 0 or len(img) + arr_size * iteration == max_size):        
        for j in range(0, len(img)):
            im = np.array(img[j])[0:500][0:500]
            im = delete_blue(im)
            im = delete_darkest(im)
            im = cv2.cvtColor(im, cv2.COLOR_HSV2BGR)
            im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
            im = cv2.Canny(im, 100, 200)
            #im = cv2.GaussianBlur(im, (11,11), 0)
            #im = cv2.cvtColor(im, cv2.COLOR_BGR2GRAY)
            #im = cv2.Sobel(im, cv2.CV_8U, 1, 0 , 1, 0, cv2.BORDER_DEFAULT)
            #im = cv2.threshold(im, 70, 255, cv2.THRESH_BINARY)[1]
            #im = compare(buoy, im)
            write(im, j)
            
        reset()
        iteration = iteration + 1
        return True
    return False

In [6]:
#Delete all pixels that are outside of a certain red/green range
def delete_blue(image):
    image = cv2.cvtColor(image, cv2.COLOR_BGR2HSV)
    image[np.logical_not(np.logical_or(np.logical_and(image[:,:,0] > 70, image[:,:,0] < 90),np.logical_or(image[:,:,0] >= 170, image[:,:,0] < 10)))] = [0,0,0]
    return(image)

In [7]:
def delete_darkest(image):
    return False

In [8]:
#for writing one instance of an image
def write(image, index):
    cv2.imwrite(out + "out" + str(index + iteration * arr_size).zfill(4) + ".jpg", image)
    return True

In [9]:
#for writing multiple instances of an image. ex. when separating an image into red and green channels
def write_rg(image, index, color_index):
    cv2.imwrite(out + "out" + str(index + iteration * arr_size).zfill(4) + "-" + str(color_index) + ".jpg", image)
    return True

In [10]:
#an extension of write_rg for multiple sets of images. ex. recording multiple filters of red and green channels of an image
def write_rg_mult(image, index, color_index, mult):
    cv2.imwrite(out + "out" + str(index + iteration * arr_size).zfill(4) + "-" + str(color_index) + "-" + str(mult) + ".jpg", image)
    return True

In [11]:
#Use SIFT to detect a given feature in an image
def compare(image1,image2):
    sift = cv2.xfeatures2d.SIFT_create()
        
    kp1, des1 = sift.detectAndCompute(image1,None)
    kp2, des2 = sift.detectAndCompute(image2,None)
    
    FLANN_INDEX_KDTREE = 1
    index_params = dict(algorithm = FLANN_INDEX_KDTREE, trees = 5)
    search_params = dict(checks=50)   # or pass empty dictionary)
    
    flann = cv2.FlannBasedMatcher(index_params,search_params)
    matches = flann.knnMatch(des1,des2,k=2)
    
    # Need to draw only good matches, so create a mask
    matchesMask = [[0,0] for i in range(len(matches))]
    
    # ratio test as per Lowe's paper
    for i,(m,n) in enumerate(matches):
        if m.distance < 0.7*n.distance:
            matchesMask[i]=[1,0]
    
    draw_params = dict(matchColor = (0,255,0),
                    singlePointColor = (255,0,0),
                    matchesMask = matchesMask,
                    flags = cv2.DrawMatchesFlags_DEFAULT)
    
    img3 = cv2.drawMatchesKnn(image1,kp1,image2,kp2,matches,None,**draw_params)
    return img3


In [12]:
def ofb_compare(image1, image2):
    orb = cv2.ORB()
    kp = orb.detect(image2, None)
    kp, des = org.compute(image2, kp)
    image2 = cv2.drawKeypoints(image2, kp, color=(255,0,0), flags=0)
    
    index_params= dict(algorithm = FLANN_INDEX_LSH, 
                       table_number = 6, 
                       key_size = 12, 
                       multi_probe_level = 1)
    search_params = 

In [13]:
#Delete the contents of the image arrays, used for preserving memory by doing operations on images in batches
def reset():
    del img[:]
    del original[:]
    return True

In [14]:
if __name__ == "__main__":
    iterator()

1
2
5
50
71
105
155
164
186
191
197
216
229
265
327
1428
1429
1445
1690
1698
1701
1708
1718
1719
1724
1738
1744
1746
1750
1751
1753
1756
1761
1767
1768
1772
1776
1799
1808
1813
1825
1828
1830
1837
1843
1848
1849
1851
1852
1853
1856
1858
1859
1860
1861
1862
1864
1866
2677
2684
2685
2686
2688
2693
2694
2696
2697
2698
2699
2700
2701
2702
